# Import Libraries

In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from symspellpy import SymSpell
from difflib import SequenceMatcher
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping

# Read Dataset

In [2]:
df = pd.read_excel("Product Matching Dataset.xlsx" , sheet_name="Dataset")
df.head(n=10)

,sku,marketplace_product_name_ar,seller_item_name,price
0,1322,استوهالت 40 مجم 14 كبسول,ESTOHALT 40 MG 14 CAP,56.5
1,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
2,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
3,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
4,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
5,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
6,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
7,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 1 شريط * 14 كبسولة,56.5
8,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم,56.5
9,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 كبسول س ج,56.5


# Preprocessing

In [3]:
stop_words = set(stopwords.words("arabic")) | set(stopwords.words("english"))

# text_column = df["seller_item_name"].astype(str)  # Ensure text format

In [4]:
def preprocessing(text):
    # Remove stopwords
    text = " ".join(word for word in text.split() if word not in stop_words)
    # Remove punctuation & special characters (Keep Arabic, English, and Numbers)
    text = re.sub(r"[^\w\s\u0600-\u06FF]", " ", text)
    # Remove Arabic diacritics (Tashkeel)
    text = re.sub(r"[\u064B-\u065F]", "", text)

    text = re.sub(r'[إأآ]', 'ا', text)
    text = re.sub(r'ى', 'ي', text)
    text = re.sub(r'ة', 'ه ', text)
    text = re.sub(r'ؤ', 'و', text)
    text = re.sub(r'ئ', 'ي', text)
    text = re.sub(r'ــ', '', text)
    text = re.sub(r"(\d+)", r" \1 ", text)
    text = re.sub(r"\b[\u0600-\u06FF]\b", "", text)
    text = re.sub(r'([\u0600-\u06FF])\1{2,}', r'\1\1', text)
    text = re.sub(r'(.)\1+', r'\1', text) # remove repetitions
    text = re.sub(r'\b\w\b', '', text)  # Removes standalone single characters
    text = re.sub(r'\b(?:سعر جديد|س جديد|س جدي|س ج|ركز)\b', '', text)# Remove specific unwanted phrases
    text = re.sub( r'مرهم|اكريم', 'كريم', text)
    text = re.sub( r'قرص|اقراص|كبسوله', 'كبسول', text)
    text = re.sub( r'اقراص|شريط|شرائط|شريطين', 'قرص', text)
    text = re.sub( r'امبولات|امبوله|حقن', 'امبول', text)
    text = re.sub( r'لبوس|لبوس اطفال|لبوس اطفال فاركو', 'اقماع للاطفال', text)
    text = re.sub(r"\s+", " ", text).strip()
    
    return text

In [5]:
df["cleaned_seller_item_name"] = df["seller_item_name"].astype(str).apply(preprocessing)
scaler = StandardScaler()
df["scaled_price"] = scaler.fit_transform(df[["price"]])
X = df[["cleaned_seller_item_name", "scaled_price"]]
Y = df["sku"]

X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size=0.2, random_state=42, stratify=Y, shuffle=True)


# Build SymSpell Dictionary

In [13]:
# word_freq_dict = {}
# for product in df["marketplace_product_name_ar"].dropna():
#     normalized_product = str(product).strip()
#     word_freq_dict[normalized_product] = word_freq_dict.get(normalized_product, 0) + 1

# # Save dictionary
# dict_file = "product_dictionary.txt"
# with open(dict_file, "w", encoding="utf-8") as f:
#     for word, freq in word_freq_dict.items():
#         f.write(f"{word} {freq}\n")

# # Initialize SymSpell
# sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
# sym_spell.load_dictionary(dict_file, term_index=0, count_index=1, encoding="utf-8")

# # 3. Apply SymSpell Correction
# def correct_text(text):
#     suggestions = sym_spell.lookup(text, verbosity=2, max_edit_distance=2)
#     return suggestions[0].term if suggestions else text

# df["corrected_seller_item_name"] = df["cleaned_seller_item_name"].apply(correct_text)

# Train Models

In [14]:
vectorizer = TfidfVectorizer(max_features=500)
X_train_tfidf = vectorizer.fit_transform(X_train["cleaned_seller_item_name"])
X_test_tfidf = vectorizer.transform(X_test["cleaned_seller_item_name"])

# Train Logistic Regression Model
logistic_model = LogisticRegression()
logistic_model.fit(X_train_tfidf, y_train)

y_pred = logistic_model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print(f"Logistic Regression Model Accuracy: {accuracy:.4f}")

Logistic Regression Model Accuracy: 0.8648


In [ ]:
# label_encoder = LabelEncoder()
# df["encoded_sku"] = label_encoder.fit_transform(df["sku"])

# # Tokenization for RNN
# num_words = 5000
# tokenizer = Tokenizer(num_words=num_words, oov_token="<OOV>")
# tokenizer.fit_on_texts(df["cleaned_seller_item_name"])
# sequences = tokenizer.texts_to_sequences(df["cleaned_seller_item_name"])

# # Padding
# max_length = max(len(seq) for seq in sequences)
# x_padded = pad_sequences(sequences, maxlen=max_length, padding="post", truncating="post")

# # Train-Test Split for RNN
# X_train, X_test, y_train, y_test = train_test_split(x_padded, df["encoded_sku"], test_size=0.3, random_state=42)

# # Build RNN Model
# rnn_model = Sequential([
#     Embedding(input_dim=num_words, output_dim=128, input_length=max_length),
#     LSTM(64, return_sequences=True),
#     Dropout(0.2),
#     LSTM(32),
#     Dropout(0.2),
#     Dense(32, activation="relu"),
#     Dense(len(label_encoder.classes_), activation="softmax")
# ])

# rnn_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
# # Early Stopping
# early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

# # Train RNN Model
# rnn_model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

# loss, accuracy = rnn_model.evaluate(X_test, y_test)
# print(f"RNN Model Accuracy: {accuracy:.4f}")


# Label Encoding for SKU
label_encoder = LabelEncoder()
df["encoded_sku"] = label_encoder.fit_transform(df["sku"])

# Tokenization for RNN
num_words = 5000
tokenizer = Tokenizer(num_words=num_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df["cleaned_seller_item_name"])
sequences = tokenizer.texts_to_sequences(df["cleaned_seller_item_name"])

# Padding
max_length = max(len(seq) for seq in sequences)
x_padded = pad_sequences(sequences, maxlen=max_length, padding="post", truncating="post")

# Train-Test Split for RNN
X_train, X_test, y_train, y_test = train_test_split(x_padded, df["encoded_sku"], test_size=0.3, random_state=42)

# Build Improved RNN Model
rnn_model = Sequential([
    Embedding(input_dim=num_words, output_dim=256, input_length=max_length),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.2),
    Bidirectional(GRU(32)),
    Dropout(0.2),
    Dense(32, activation="relu"),
    Dense(len(label_encoder.classes_), activation="softmax")
])

rnn_model.compile(loss="sparse_categorical_crossentropy", optimizer=Adam(learning_rate=0.0005), metrics=["accuracy"])

# Early Stopping
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

# Train RNN Model
rnn_model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

loss, accuracy = rnn_model.evaluate(X_test, y_test)
print(f"RNN Model Accuracy: {accuracy:.4f}")


Logistic Regression Model Accuracy: 0.8648


C:\Users\hassa\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/100
1828/1828 ━━━━━━━━━━━━━━━━━━━━ 44s 21ms/step - accuracy: 0.1221 - loss: 4.7405 - val_accuracy: 0.8558 - val_loss: 1.0872
Epoch 2/100
1828/1828 ━━━━━━━━━━━━━━━━━━━━ 39s 22ms/step - accuracy: 0.8361 - loss: 0.9156 - val_accuracy: 0.9420 - val_loss: 0.2969
Epoch 3/100
1828/1828 ━━━━━━━━━━━━━━━━━━━━ 39s 22ms/step - accuracy: 0.9329 - loss: 0.3006 - val_accuracy: 0.9500 - val_loss: 0.2200
Epoch 4/100
1828/1828 ━━━━━━━━━━━━━━━━━━━━ 39s 21ms/step - accuracy: 0.9475 - loss: 0.1839 - val_accuracy: 0.9501 - val_loss: 0.2161
Epoch 5/100
1828/1828 ━━━━━━━━━━━━━━━━━━━━ 39s 21ms/step - accuracy: 0.9503 - loss: 0.1493 - val_accuracy: 0.9528 - val_loss: 0.2165
Epoch 6/100
1828/1828 ━━━━━━━━━━━━━━━━━━━━ 39s 22ms/step - accuracy: 0.9564 - loss: 0.1233 - val_accuracy: 0.9559 - val_loss: 0.2106
Epoch 7/100
1828/1828 ━━━━━━━━━━━━━━━━━━━━ 39s 22ms/step - accuracy: 0.9583 - loss: 0.1090 - val_accuracy: 0.9558 - val_loss: 0.2159
Epoch 8/100
1828/1828 ━━━━━━━━━━━━━━━━━━━━ 39s 21ms/step - accuracy: 

In [ ]:
def predict_sku(medicine_name, threshold=0.5):
    # Preprocess the input name
    cleaned_text = preprocessing(medicine_name)
    
    # Convert to sequence
    seq = tokenizer.texts_to_sequences([cleaned_text])
    padded_seq = pad_sequences(seq, maxlen=max_length, padding="post", truncating="post")

    # Get model predictions
    predictions = rnn_model.predict(padded_seq)[0]
    
    # Get highest probability SKU
    max_prob = np.max(predictions)
    predicted_label = np.argmax(predictions)

    # If confidence is low, return "unknown"
    if max_prob < threshold:
        return "unknown"

    # Convert label back to SKU
    return label_encoder.inverse_transform([predicted_label])[0]

# # Example Predictions
# print(predict_sku("بانادول اكسترا"))  # Should return the correct SKU or "unknown"
# print(predict_sku("شوجارلو بلس 50/1000  30"))  # Should return "unknown"
# print(predict_sku("GLIPTUS PLUS 50/1000 MG 30 TAB"))  # Should return "unknown"


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1469
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
116


In [15]:
probabilities = rnn_model.predict(X_test)

confidence_scores = np.max(probabilities, axis=1)

predicted_indices = np.argmax(probabilities, axis=1)

predicted_labels = label_encoder.inverse_transform(predicted_indices)

confidence_threshold = 0.85

test_results = []

for i in range(len(X_test)):
    confidence = confidence_scores[i]
    predicted_class = predicted_labels[i]

    if confidence < confidence_threshold:
        predicted_class = "Unknown"

    test_results.append({
        'Predicted': predicted_class,
        'Confidence': f"{confidence:.2f}"
    })

temp_df = pd.DataFrame(test_results)

temp_df.head()
df.to_csv("predict.csv" , index=False)


784/784 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step
